In [2]:
#Assignment 1 - Data Science for Business 2

import pandas as pd
import numpy as np
from gurobipy import *

# Import data:
risk = pd.read_csv("RiskDistribution.txt", delimiter="\t", names=["p", "N"])
p0 = np.array(risk["p"] / 100)
N = risk["N"]

# Generate problem parameters:

# Test characteristics: sensitivity & specificity. A1 = test 1, A2 = test 2
# Columns: result +, result -
# Rows: CHD Event, No CHD Event

A1 = np.array([[0.75, 0.25], [0.15, 0.85]])
A2 = np.array([[0.95, 0.05], [0.05, 0.95]])

# For testing & debugging:
#A1 = np.array([[0.01, 0.99], [0.99, 0.01]])
#A2 = np.array([[0.95, 0.05], [0.05, 0.95]])

# Test costs [test1, test2]
TestC = np.array([500.0, 1000.0])

# Health outcomes
# Rows: disease, no disease
# Columns: No treatment, Treatment
H = np.array([[6.9, 7.1], [7.8, 7.7] ])

# Treatment costs
# Rows: CHD Event, No CHD Event
# Columns: No treatment, Treatment
TreatmentC = np.array([[15000.0, 12000.0], [0.0, 2000.0]])

# WTP threshold
# wtp is the same as 'lambda' in the problem description
wtp = 50000.0


### Gurobi implementation

# Constructs a problem specific Gurobi model corresponding to the decision tree (without objective function)

m = Model("CHD optimizer") # creates an empty model

decisionMap = {}# dictionary for holding information states (as keys) and related decision variables

# The variables are added to a dictionary, that has as keys a tuple consisting of the relevant 'history' of the decision
# and the decision type (test or treat)

# This allows connecting decision making situations to the corresponding variables later on
# indexing note: test 1: key value = 0 / test 2: key value = 1

# The keys are tuples in format (history tuple, testtype) where history tuple is in
# format ((1 round test, test result), (2nd round test test, result))

# Example: tuple ((('no test',), ('no test',)) 'test) means 'no tests performed, test decision in question'
# and (((0, 1), (1, 0)), 'treat') means 'first test was test 1 with negative result, ...
# 2nd test was test 2 with positive result, treatment decision to be made'

# Each Gurobi / Python decision variable (e.g. 'test_0') contains a certain amount of binary LP model decision variables
# corresponding to the choices available:
# E.g. test_0 contains three decision variables corresponding to 'don't test' / 'do test 1' / 'do test 2'

# Note: these are here added one by one to explicitly show what happens but in more complex cases the code
# should probably generate the variables more dynamically.

# Note: this is only one way to handle linking variables to situations. In some cases using addVars() capability to
# construct variables using indice vectors might be very useful
# (e.g. x = addVars([0, 1], ['+', '-']) would create 4 variables that could be accessed
# using  x[0, '+'], x[0, '-'], x[1, '+'], x[1, '-'])
# In this case however, this would lead to unused variables.

# Variables x1, x2: Don't test / do test 1 
test_0 = m.addVars(2, vtype=GRB.BINARY, name="first_test")
decisionMap[((('no test',),('no test',)), 'test')] = test_0

# Variables x3, x4: After not testing, don't treat / treat
treat_0 = m.addVars(2, vtype=GRB.BINARY, name='treat_after_noTest')
decisionMap[((('no test',),('no test',)), 'treat')] = treat_0

# Variables x5, x6: After test 1 with result +, don't test / do test 2
test_1pos = m.addVars(2, vtype=GRB.BINARY, name='test_after_test1+')
decisionMap[(((0, 0),('no test',)), 'test')] =test_1pos

# Variables x7, x8: After test 1 with result -, don't test / do test 2
test_1neg = m.addVars(2, vtype=GRB.BINARY, name='test_after_test1-')
decisionMap[(((0, 1),('no test',)), 'test')] = test_1neg

# Variables x9, x10: After test 1 with result +, don't treat / treat
treat_1pos = m.addVars(2, vtype=GRB.BINARY, name='treat_after_test1+')
decisionMap[(((0, 0),('no test',)), 'treat')] =treat_1pos

# Variables x11, x12: After test 1 with result -, don't treat / treat
treat_1neg = m.addVars(2, vtype=GRB.BINARY, name='treat_after_test1-')
decisionMap[(((0, 1),('no test',)), 'treat')] =treat_1neg

# Variables x13, x14: After test 1 with result + and test 2 with result +, don't treat / treat
treat_1pos2pos = m.addVars(2, vtype=GRB.BINARY, name='treat_after_test1+_test2+')
decisionMap[(((0, 0), (1, 0)), 'treat')] = treat_1pos2pos

# Variables x15, x16: After test 1 with result + and test 2 with result -, don't treat / treat
treat_1pos2neg = m.addVars(2, vtype=GRB.BINARY, name='treat_after_test1+_test2-')
decisionMap[(((0, 0), (1, 1)), 'treat')] = treat_1pos2neg

# Variables x17, x18: After test 1 with result - and test 2 with result +, don't treat / treat
treat_1neg2pos = m.addVars(2, vtype=GRB.BINARY, name='treat_after_test1-_test2+')
decisionMap[(((0, 1), (1, 0)), 'treat')] = treat_1neg2pos

# Variables x19, x20: After test 1 with result - and test 2 with result -, don't treat / treat
treat_1neg2neg = m.addVars(2, vtype=GRB.BINARY, name='treat_after_test1-_test2-')
decisionMap[(((0, 1), (1, 1)), 'treat')] = treat_1neg2neg

# Constraints: These follow the math formulation as described in lecture slides

# Exactly one testing option 𝑗∈{0,1} must be selected in the first stage (no test, test 1)
m.addConstr(sum(test_0[i] for i in range(2)) == 1)

# If no tests are carried out (𝑗=0), then exactly one treatment option 𝑘∈{0,1} must be selected (no treatment/treatment)
m.addConstr(sum(treat_0[i] for i in range(2)) == test_0[0])

# If test 1 is carried out in the first stage, then given any test result 𝑚∈{+,−} exactly one testing option
# ...𝑗∈{0,2} (no test, test 2) must be selected for the second stage
m.addConstr(sum(test_1pos[i] for i in range(2)) == test_0[1])
m.addConstr(sum(test_1neg[i] for i in range(2)) == test_0[1])

# If test 1 with result 𝑚 is followed by no test, then exactly one treatment option 𝑘∈{0,1}
# ... must be selected (no treatment / treatment)
m.addConstr(sum(treat_1pos[i] for i in range(2)) == test_1pos[0])
m.addConstr(sum(treat_1neg[i] for i in range(2)) == test_1neg[0])

# If test 1 is followed by test 2, then exactly one treatment option 𝑘∈{0,1} (no treatment / treatment) must be
# ... selected given any combination of test results.
m.addConstr(sum(treat_1pos2pos[i] for i in range(2)) == test_1pos[1])
m.addConstr(sum(treat_1pos2neg[i] for i in range(2)) == test_1pos[1])
m.addConstr(sum(treat_1neg2pos[i] for i in range(2)) == test_1neg[1])
m.addConstr(sum(treat_1neg2neg[i] for i in range(2)) == test_1neg[1])


# This function solves an optimization model m, with objective function f
# (Using a function to do this helps with updating the model)
def solve_objective(m, f):
    # Inputs:
    #   m: Gurobi model
    #   f: objective function for the Gurobi model
    # Outputs:
    #   Optimal solution and objective value

    # Initialization of variables for storing results:
    obj_value = []
    var_values = []

    try:

        # Set objective function and select to minimize:
        m.setObjective(f, GRB.MAXIMIZE)

        m.update()

        # Optimize, i.e. solve the model
        m.optimize()

        # This prints the model description to an .lp file (could be used e.g. for debugging or to export the model)
        m.write('CHDmodel.lp')

        # get results into var_values
        var_values = m.getVars()

        # get objective value
        obj_value = m.objVal

        # Just print the results:
        for v in var_values:
            print('%s %g' % (v.varName, v.x))

        print('Obj: %g' % m.objVal)


    except GurobiError as e:
        print('Error code ' + str(e.errno) + ": " + str(e))

    except AttributeError:
        print('Encountered an attribute error')

    return obj_value, var_values


# This function runs the case (i.e. optimizes for each initial prior probability segment
def run():

    # Initialize result variables:
    X = np.zeros((101,20))  # Initialize optimal strategy matrix
    optValues = np.zeros((101, 1))

    # Initialize share of decisions based on prior risk in optimal solution  (don't treat, test1, treat)
    sharesPrior = np.zeros((101, 3))

    # Initialize share of decisions after 1st decision = test 1 in optimal solution (don't treat, test2, treat)
    sharesTest1 = np.zeros((101, 3))

    # Initialize share of decisions after results of test 1 followed by test 2 in optimal solution  (don't treat, treat)
    sharesTests12 = np.zeros((101, 2))

    shares = np.zeros((101, 8))  # Initialize all

    # These will store the objective function parameter components:
    costVector = {}
    healthVector = {}

    # Arrays for storing optimal cost and health values
    optCosts =  np.zeros((101, 1))
    optHealths = np.zeros((101, 1))

    # Model constructed and solved for each prior risk group separately. Only objective function updated between groups:
    for i in range(0, 101):

        # Probabilities of results + and - (column) for tests 1 in the first stage:
        PR1 = np.array( [p0[i] * A1[0, 0] + (1 - p0[i]) * A1[1, 0], 1 - (p0[i] * A1[0, 0] + (1 - p0[i]) * A1[1, 0])] )

        # Matrix of updated probabilities of disease given results + / - (col's) from tests 1 in first stage
        P1 = np.array( [p0[i]*A1[0, 0]/PR1[0], p0[i]*A1[0, 1]/PR1[1]] ) 

        # Matrix of the probabilities of results +/- for the second-stage test
        #    (columns) given result +/- from the first stage test - rows 1 and 2
        #    corresponding to test 1 in the first stage, rows 3 and 4 to test 2 in

        PR2 = np.array([ [P1[0] * A2[0, 0] + (1 - P1[0]) * A2[1, 0], P1[0] * A2[0, 1] + (1 - P1[0]) * A2[1, 1] ], [ P1[1] * A2[0, 0]+(1-P1[1]) * A2[1, 0], P1[1] * A2[0, 1]+(1-P1[1]) * A2[1, 1]] ]) 

        # Matrix of updated disease probabilities for results + and - from test 1 (rows) and test 2 (columns);
        # (ordering does not matter)
        P2 = np.array([ [P1[0]*A2[0, 0]/PR2[0,0], P1[0]*A2[0, 1]/PR2[0,1]], [P1[1]*A2[0, 0]/PR2[1,0], P1[1]*A2[0, 1]/PR2[1,1]] ])

        # Variable related costs:
        cost = 0
        health = 0

        # Print out probability matrices for debugging
        # print("PR1: ", PR1)
        # print("P1: ", P1)
        # print("PR2: ", PR2)
        # print("P2: ", P2)
        
        # Following loop loops through each decision variable and calculates a parameter for it.
        # Parameter components (health, cost) are also saved separately into a dictionary to allow
        # calculation of descriptive information for the solution (such as e.g. costs per priori group)

        # indexing note: test 1: key value = 0 / test 2: key value = 0
        for k in decisionMap: # go through all decision situations

            l = 2 - k[0].count(('no test',)) # number of test decisions taken
            print("l: ", l, " k[0]: ", k[0])
            nd = len(decisionMap[k]) # number of decisions per gurobi variable

            if l == 0: # no tests done, either decide to test or to treat.

                # calculate the cost and health parameters for the decision variable
                for j in range(0, nd):

                    if k[-1] == "treat":
                        # add costs from the decision and its outcomes (either has CHD or not)
                        costVector[(k, j)] = (p0[i] * TreatmentC[0, j] + (1-p0[i]) * TreatmentC[1, j])
                        cost += costVector[(k, j)] * decisionMap[k][j]

                        healthVector[(k, j)] = (p0[i] * H[0, j] + (1-p0[i]) * H[1, j])
                        health += healthVector[(k, j)] * decisionMap[k][j]

                    if k[-1] == "test":
                        if not j == 0:
                            costVector[(k, j)] = TestC[0]

                        else:
                            costVector[(k, j)] = 0
                        cost += costVector[(k, j)] * decisionMap[k][j]



            if l == 1: # one test done
                firstTest = k[0][0][0]
                firstResult = k[0][0][1]

                # calculate the cost and health parameters for the decision variable
                for j in range(0, nd):
                    if k[-1] == "treat":
                        costVector[(k, j)] = PR1[firstResult] * (P1[ firstResult ] * TreatmentC[0, j] + (1-P1[ firstResult ]) * TreatmentC[1, j])
                        cost += costVector[(k, j)] * decisionMap[k][j]

                        healthVector[(k, j)] = PR1[firstResult] * (P1[ firstResult ] * H[0, j] + (1-P1[ firstResult ]) * H[1, j])
                        health += healthVector[(k, j)] * decisionMap[k][j]

                    if k[-1] == "test":
                        if j == 0:
                            costVector[(k, j)] = 0
                        else:
                            costVector[(k, j)] = PR1[firstResult] * TestC[1]

                        cost += costVector[(k, j)] * decisionMap[k][j]

                    # add (paramter * decision variable) component to objective function
                    
            if l == 2: # two tests done

                firstTest = k[0][0][0]
                firstResult = k[0][0][1]
                secondTest = k[0][1][0]
                secondResult = k[0][1][1]
                
                if firstTest == 0: # first test = test 1
                    test1_i = k[0][0][1] # test1_i picks the value from the correct result line from P2 based on key
                    test2_i = k[0][1][1] # test2_i picks the value from the correct result column from P2 based on key
                else:
                    test2_i = k[0][0][1]
                    test1_i = k[0][1][1]

                # calculate the cost and health parameters for the decision variable
                for j in range(0, nd):
                    if k[-1] == "treat":

                        costVector[(k, j)] = PR1[firstResult] * PR2[firstResult, secondResult] * (P2[ test1_i, test2_i ] * TreatmentC[0, j] + (1-P2[ test1_i, test2_i ]) * TreatmentC[1, j])
                        cost += costVector[(k, j)] * decisionMap[k][j]
                        
                        healthVector[(k, j)] = PR1[firstResult]* PR2[firstResult, secondResult] * (P2[ test1_i, test2_i ] * H[0, j] + (1-P2[ test1_i, test2_i ]) * H[1, j])
                        health += healthVector[(k, j)] * decisionMap[k][j]

                    # add (paramter * decision variable) component to objective function

        # objective function:
        f = wtp * health - cost

        # Solve the model with the objective function:
        optF, optX = solve_objective(m, f)

        # Save found objective function value into optF:
        optValues[i] = optF

        # Create a binary vector from solution and save it into X
        var_ind = 0
        for x in optX:

            X[i, var_ind] = x.X
            var_ind += 1

        # Descriptive information about the solution - can be used to calculate how many patients end up in
        # different branched of the tree:
        sharesPrior[i,:]=[X[i, 2], X[i, 1], X[i, 3]]
        sharesTest1[i, :]=[PR1[0] * X[i, 8] + PR1[1] * X[i, 10], PR1[0] * X[i, 5] + PR1[1] * X[i, 7], PR1[0] * X[i, 9] + PR1[1] * X[i, 11]]
        sharesTests12[i, :]=[PR1[0] * PR2[0, 0] * X[i, 12] + PR1[0] * PR2[0, 1] * X[i, 14] + PR1[1] * PR2[1, 0] * X[i, 16] + PR1[1] * PR2[1, 1] * X[i, 18],
            PR1[0] * PR2[0, 0] * X[i, 13] + PR1[0] * PR2[0, 1] * X[i, 15] + PR1[1] * PR2[1, 0] * X[i, 17] + PR1[1] * PR2[1, 1] * X[i, 19]]
        shares[i, :]=np.concatenate( (sharesPrior[i, :], sharesTest1[i, :], sharesTests12[i, :]) )

        optCost = 0
        optHealth = 0

        # extract costs and health effects
        for k in decisionMap:
            for j in range(len(decisionMap[k])):
                try:
                    optCost += decisionMap[k][j].X * costVector[(k, j)]
                except KeyError:
                    optCost += 0
                try:
                    optHealth += decisionMap[k][j].X * healthVector[(k, j)]
                except KeyError:
                    optHealth += 0

        optCosts[i] = optCost
        optHealths[i] = optHealth

    # Segments for first-stage decisions
    try:
        notTreated = [np.min(np.where(sharesPrior[:,0]==1)[0]), np.max(np.where(sharesPrior[:,0]==1)[0])]
    except ValueError:
        notTreated = "no segment"
    try:
        tested1 = [np.min(np.where(sharesPrior[:,1]==1)[0]), np.max(np.where(sharesPrior[:,1]==1)[0])]
    except ValueError:
        tested1 = "no segment"
    try:
        treated = [np.min(np.where(sharesPrior[:,2]==1)[0]), np.max(np.where(sharesPrior[:,2]==1)[0])]
    except ValueError:
        treated = "no segment"


    print("\nSegmentation for first decisions: \n NOTE: prints just the first and last found instance - overlaps in printout may be possible. See result files for details. \n\n Not treated: ", notTreated, "\n Tested 1: ", tested1, "\n treated: ",treated)

    return optValues, X, shares, optCosts, optHealths
    
    
optValues, solutions, shares, optCosts, optHealths = run()

totalCosts = np.dot(N, optCosts)
totalHealth = np.dot(N, optHealths)


# print results to a csv file:
p = np.expand_dims(p0, axis=1)
N = np.expand_dims(N, axis=1)
healthPerPatient = np.expand_dims(optCosts, axis=1)
costsPerPatient = np.expand_dims(optHealths, axis=1) 


patientsInBranches = shares * N

# create headers for result files:

result_header = 'prior, E[U], Health, Costs, prior: dont test, prior: test 1, no test: dont treat, no test: treat, test1+: no 2nd test, test1+: test2, test1-:no 2nd test, test1-:test2, test1+ (no 2nd test): dont treat, test1+ (no 2nd test): treat, test1- (no 2nd test): dont treat, test1- (no 2nd test): treat, test1+test2+: dont treat, test1+test2+: treat, test1+test2-: dont treat, test1+test2-: treat, test1-test2+: dont treat, test1-test2+: treat, test1-test2-: dont treat, test1-test2-: treat'

share_header = 'prior: dont treat,	prior: test 1,	prior: treat,	test1: dont treat,	test1: test2,	test1: treat, test1-test2: dont treat, test1-test2: treat'

results = np.concatenate((p, optValues, optHealths, optCosts, solutions), axis=1)
np.savetxt('CHD_optimization_resultsFinal_wtp' + str(int(wtp)) + '.csv', results, delimiter=',', fmt='%10.5f', header=result_header, comments='') # contains solutions and objective values
np.savetxt('CHD_optimization_sharesFinal_wtp' + str(int(wtp)) + '.csv', shares, delimiter=',', fmt='%10.5f', header=share_header, comments='') # contains split of patients into 'treatment segments'
np.savetxt('CHD_optimization_peopleFinal_wtp' + str(int(wtp)) + '.csv', patientsInBranches, delimiter=',',  fmt='%10.5f', header=share_header, comments='') # number of patients in branches of tree

# Print total health effects and costs

print("\nTotal costs: ", totalCosts)
print("Total health effects: ", totalHealth)


l:  0  k[0]:  (('no test',), ('no test',))
l:  0  k[0]:  (('no test',), ('no test',))
l:  1  k[0]:  ((0, 0), ('no test',))
l:  1  k[0]:  ((0, 1), ('no test',))
l:  1  k[0]:  ((0, 0), ('no test',))
l:  1  k[0]:  ((0, 1), ('no test',))
l:  2  k[0]:  ((0, 0), (1, 0))
l:  2  k[0]:  ((0, 0), (1, 1))
l:  2  k[0]:  ((0, 1), (1, 0))
l:  2  k[0]:  ((0, 1), (1, 1))
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 10 rows, 20 columns and 29 nonzeros
Model fingerprint: 0xc08900fe
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 390000.00000
Presolve removed 10 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 390000 

Optim

l:  1  k[0]:  ((0, 0), ('no test',))
l:  1  k[0]:  ((0, 1), ('no test',))
l:  1  k[0]:  ((0, 0), ('no test',))
l:  1  k[0]:  ((0, 1), ('no test',))
l:  2  k[0]:  ((0, 0), (1, 0))
l:  2  k[0]:  ((0, 0), (1, 1))
l:  2  k[0]:  ((0, 1), (1, 0))
l:  2  k[0]:  ((0, 1), (1, 1))
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 10 rows, 20 columns and 29 nonzeros
Model fingerprint: 0x2c92e345
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 387000

Presolve removed 10 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 387000 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.870000000000e+05, 

Optimize a model with 10 rows, 20 columns and 29 nonzeros
Model fingerprint: 0xb432830a
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 384169

Presolve removed 10 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 384169 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.841690000000e+05, best bound 3.841690000000e+05, gap 0.0000%
first_test[0] 0
first_test[1] 1
treat_after_noTest[0] 0
treat_after_noTest[1] 0
test_after_test1+[0] 0
test_after_test1+[1] 1
test_after_test1-[0] 1
test_after_test1-[1] 0
treat_after_test1+[0] 0
treat_after_test1+[1] 0
treat_after_test1-[0] 1
treat_after_test1-[1] 0
treat_after_test

Optimize a model with 10 rows, 20 columns and 29 nonzeros
Model fingerprint: 0x41fd6e0d
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 381605

Presolve removed 10 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.20 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 381605 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.816047500000e+05, best bound 3.816047500000e+05, gap 0.0000%
first_test[0] 0
first_test[1] 1
treat_after_noTest[0] 0
treat_after_noTest[1] 0
test_after_test1+[0] 0
test_after_test1+[1] 1
test_after_test1-[0] 1
test_after_test1-[1] 0
treat_after_test1+[0] 0
treat_after_test1+[1] 0
treat_after_test1-[0] 1
treat_after_test1-[1] 0
treat_after_test

Optimize a model with 10 rows, 20 columns and 29 nonzeros
Model fingerprint: 0xc6a85cc5
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 379040

Presolve removed 10 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 379040 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.790405000000e+05, best bound 3.790405000000e+05, gap 0.0000%
first_test[0] 0
first_test[1] 1
treat_after_noTest[0] 0
treat_after_noTest[1] 0
test_after_test1+[0] 0
test_after_test1+[1] 1
test_after_test1-[0] 1
test_after_test1-[1] 0
treat_after_test1+[0] 0
treat_after_test1+[1] 0
treat_after_test1-[0] 1
treat_after_test1-[1] 0
treat_after_test

...

Optimize a model with 10 rows, 20 columns and 29 nonzeros
Model fingerprint: 0x0d3f83b7
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+02, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 351131

Presolve removed 10 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 351131 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.511305000000e+05, best bound 3.511305000000e+05, gap 0.0000%
first_test[0] 0
first_test[1] 1
treat_after_noTest[0] 0
treat_after_noTest[1] 0
test_after_test1+[0] 1
test_after_test1+[1] 0
test_after_test1-[0] 0
test_after_test1-[1] 1
treat_after_test1+[0] 0
treat_after_test1+[1] 1
treat_after_test1-[0] 0
treat_after_test1-[1] 0
treat_after_test

Optimize a model with 10 rows, 20 columns and 29 nonzeros
Model fingerprint: 0x06c085df
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 349000

Presolve removed 10 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.14 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 349000 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.490000000000e+05, best bound 3.490000000000e+05, gap 0.0000%
first_test[0] 1
first_test[1] 0
treat_after_noTest[0] 0
treat_after_noTest[1] 1
test_after_test1+[0] 0
test_after_test1+[1] 0
test_after_test1-[0] 0
test_after_test1-[1] 0
treat_after_test1+[0] 0
treat_after_test1+[1] 0
treat_after_test1-[0] 0
treat_after_test1-[1] 0
treat_after_test

Optimize a model with 10 rows, 20 columns and 29 nonzeros
Model fingerprint: 0x087f0049
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 347000

Presolve removed 10 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 347000 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.470000000000e+05, best bound 3.470000000000e+05, gap 0.0000%
first_test[0] 1
first_test[1] 0
treat_after_noTest[0] 0
treat_after_noTest[1] 1
test_after_test1+[0] 0
test_after_test1+[1] 0
test_after_test1-[0] 0
test_after_test1-[1] 0
treat_after_test1+[0] 0
treat_after_test1+[1] 0
treat_after_test1-[0] 0
treat_after_test1-[1] 0
treat_after_test

Optimize a model with 10 rows, 20 columns and 29 nonzeros
Model fingerprint: 0xee333b06
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 345000

Presolve removed 10 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 345000 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.450000000000e+05, best bound 3.450000000000e+05, gap 0.0000%
first_test[0] 1
first_test[1] 0
treat_after_noTest[0] 0
treat_after_noTest[1] 1
test_after_test1+[0] 0
test_after_test1+[1] 0
test_after_test1-[0] 0
test_after_test1-[1] 0
treat_after_test1+[0] 0
treat_after_test1+[1] 0
treat_after_test1-[0] 0
treat_after_test1-[1] 0
treat_after_test

Optimize a model with 10 rows, 20 columns and 29 nonzeros
Model fingerprint: 0x12a78003
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 343000

Presolve removed 10 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 343000 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.430000000000e+05, best bound 3.430000000000e+05, gap 0.0000%
first_test[0] 1
first_test[1] 0
treat_after_noTest[0] 0
treat_after_noTest[1] 1
test_after_test1+[0] 0
test_after_test1+[1] 0
test_after_test1-[0] 0
test_after_test1-[1] 0
treat_after_test1+[0] 0
treat_after_test1+[1] 0
treat_after_test1-[0] 0
treat_after_test1-[1] 0
treat_after_test